<a href="https://colab.research.google.com/github/SIDHARTH06/BERT-MODEL-QN-ANSWERING/blob/master/BertMode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 29.0 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 636 kB 59.0 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [63]:
import pandas as pd
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import torch

In [ ]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer:\n{}".format(answer.capitalize()))

In [ ]:
question_answer("1 The most beautiful humming birds are found in the West Indies and South America. The crest of the tiny head of one of these shines like a sparkling crown of coloured light. The shades of colour that adorn its breast are equally brilliant. As the bird flits from one object to another, it looks more like a bright flash of sunlight than it does like a living being.But, you ask, why are they called humming birds? It is because they make a soft, humming noise by the rapid motion of their wings—a motion so rapid, that as they fly, you can hardly see that they have wings. One day when walking in the woods, I found the nest of one of the smallest humming birds. It was about half the size of a very small hen and lays egg, and it was attached to a twig no thicker than a steel knitting needle. It seemed to have been made of cotton fibres and was covered with the softest bits of leaf and bark. It had two eggs in it, and each was about as large as a small sugarplum. When you approach the spot where one of these birds has built its nest, it is necessary to be careful. The mother bird will dart at you and try to peck your eyes. Its sharp beak may hurt your eyes most severely and even destroy your sight. The poor little thing knows no other way of defending its young, and instinct teaches it that you might carry off its nest if you find it.","Why are they called humming birds?")


Predicted answer:
Because they make a soft , humming noise by the rapid motion of their wings
